### Background
* The dataset from [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators) has a main problem. The data does not specify the type of diabetes, whether it's type 1 or type 2. [This notebook](https://www.kaggle.com/code/alexteboul/diabetes-health-indicators-dataset-notebook) shows how the data provider cleaned the data.

### Purpose
* Prevent the loss of information due to down-sampling
* Exclude the cases due to pregnancy instead of considering them as no diabetes
* Consider pre-diabetes to diabetes as positive cases. This is because the data provider considered no diabetes and pre-diabetes as 1 group. This contradicted to the description of dataset. 

### Import packages

In [1]:
import pandas as pd

### Import data

In [2]:
df0 = pd.read_csv("./hide/raw data/BRFSS-2015.csv")

### Select columns

In [3]:
columns = ['DIABETE3', '_RFHYPE5', 'TOLDHI2', '_CHOLCHK', '_BMI5', 'SMOKE100', 'CVDSTRK3', '_MICHD', '_TOTINDA', '_FRTLT1', '_VEGLT1', '_RFDRHV5', 'HLTHPLN1', 'MEDCOST', 'GENHLTH', 'MENTHLTH', 'PHYSHLTH', 'DIFFWALK', 'SEX', '_AGEG5YR', 'EDUCA', 'INCOME2']
df = df0[columns]

In [4]:
df.shape

(441456, 22)

### Select rows

In [5]:
# condition = ~(df.DIABETE3.isin([7,9])) & ~(df._RFHYPE5.isin([9])) & ~(df.TOLDHI2.isin([7,9])) & ~(df._CHOLCHK.isin([9])) &~(df.SMOKE100.isin([7,9])) & ~(df.CVDSTRK3.isin([7,9])) & ~(df._TOTINDA.isin([9])) &  ~(df._FRTLT1.isin([9])) & ~(df._VEGLT1.isin([9])) & ~(df._RFDRHV5.isin([9])) & ~(df.HLTHPLN1.isin([7,9])) &  ~(df.MEDCOST.isin([7,9])) & ~(df.GENHLTH.isin([7,9])) & ~(df.MENTHLTH.isin([77,99])) & ~(df.PHYSHLTH.isin([77,99])) & ~(df.DIFFWALK.isin([7,9])) & ~(df._AGEG5YR.isin([14])) & ~(df.EDUCA.isin([9])) & ~(df.INCOME2.isin([77,99]))  # check the data provider
condition = ~(df.DIABETE3.isin([2,7,9])) & ~(df._RFHYPE5.isin([9])) & ~(df.TOLDHI2.isin([7,9])) & ~(df._CHOLCHK.isin([9])) &~(df.SMOKE100.isin([7,9])) & ~(df.CVDSTRK3.isin([7,9])) & ~(df._TOTINDA.isin([9])) &  ~(df._FRTLT1.isin([9])) & ~(df._VEGLT1.isin([9])) & ~(df._RFDRHV5.isin([9])) & ~(df.HLTHPLN1.isin([7,9])) &  ~(df.MEDCOST.isin([7,9])) & ~(df.GENHLTH.isin([7,9])) & ~(df.MENTHLTH.isin([77,99])) & ~(df.PHYSHLTH.isin([77,99])) & ~(df.DIFFWALK.isin([7,9])) & ~(df._AGEG5YR.isin([14])) & ~(df.EDUCA.isin([9])) & ~(df.INCOME2.isin([77,99]))
df = df[condition]

In [6]:
# df.DIABETE3 = df.DIABETE3.replace({2:0, 3:0, 1:2, 4:1}) # check the data provider
df.DIABETE3 = df.DIABETE3.replace({3:0, 4:1, 1:2})
df._RFHYPE5 = df._RFHYPE5.replace({1:0, 2:1})
df.TOLDHI2 = df.TOLDHI2.replace({2:0})
df._CHOLCHK = df._CHOLCHK.replace({3:0,2:0})
df._BMI5 = df._BMI5.div(100).round(0)
df.SMOKE100 = df.SMOKE100.replace({2:0})
df.CVDSTRK3 = df.CVDSTRK3.replace({2:0})
df._MICHD = df._MICHD.replace({2:0})
df._TOTINDA = df._TOTINDA.replace({2:0})
df._FRTLT1 = df._FRTLT1.replace({2:0})
df._VEGLT1 = df._VEGLT1.replace({2:0})
df._RFDRHV5 = df._RFDRHV5.replace({1:0, 2:1})
df.HLTHPLN1 = df.HLTHPLN1.replace({2:0})
df.MEDCOST = df.MEDCOST.replace({2:0})
df.MENTHLTH = df.MENTHLTH.replace({88:0})
df.PHYSHLTH = df.PHYSHLTH.replace({88:0})
df.DIFFWALK = df.DIFFWALK.replace({2:0})
df.SEX = df.SEX.replace({2:0})

In [7]:
df.shape

(294938, 22)

In [8]:
df.DIABETE3.value_counts(dropna=False)

DIABETE3
0.0    251882
2.0     37955
1.0      5098
NaN         3
Name: count, dtype: int64

### Create Binary Dataset for diabetes vs. no diabetes

In [9]:
df_binary = df.copy()
df_binary = df_binary.dropna()
print("After removing missing records:")
print(df_binary.DIABETE3.value_counts(dropna=False))
df_binary.DIABETE3 = df_binary.DIABETE3.replace({2:1})
print("After further grouping pre-diabetes and diabetes:")
df_binary.DIABETE3.value_counts(dropna=False)

After removing missing records:
DIABETE3
0.0    211725
2.0     35346
1.0      4631
Name: count, dtype: int64
After further grouping pre-diabetes and diabetes:


DIABETE3
0.0    211725
1.0     39977
Name: count, dtype: int64

#### Rename columns

In [10]:
column_name = {'DIABETE3':'Diabetes_binary', 
               '_RFHYPE5':'HighBP',  
               'TOLDHI2':'HighChol', 
               '_CHOLCHK':'CholCheck', 
               '_BMI5':'BMI', 
               'SMOKE100':'Smoker', 
               'CVDSTRK3':'Stroke', 
               '_MICHD':'HeartDiseaseorAttack', 
               '_TOTINDA':'PhysActivity', 
               '_FRTLT1':'Fruits', 
               '_VEGLT1':"Veggies", 
               '_RFDRHV5':'HvyAlcoholConsump', 
               'HLTHPLN1':'AnyHealthcare', 
               'MEDCOST':'NoDocbcCost', 
               'GENHLTH':'GenHlth', 
               'MENTHLTH':'MentHlth',
               'PHYSHLTH':'PhysHlth', 
               'DIFFWALK':'DiffWalk', 
               'SEX':'Sex', 
               '_AGEG5YR':'Age', 
               'EDUCA': 'Education',
               'INCOME2':'Income'}
df_binary = df_binary.rename(columns = column_name)

### Export data

In [11]:
df_binary.to_csv("./data/BRFSS-2015_binary.csv", index=False)